# Chat: Solving the memory problem

Recall the overall workflow for retrieval augmented generation (RAG):

![](images/lesson-2-overview.png)

We discussed `Document Loading` and `Splitting` as well as `Storage` and `Retrieval`.

We then showed how `Retrieval` can be used for output generation in Q+A using `RetrievalQA` chain.

In [1]:
# pip install panel

In [2]:
import os
import openai

import panel as pn  # GUI
pn.extension()

openai.api_key  = os.environ['OPENAI_API_KEY']

The code below was added to assign the openai LLM version filmed until it is deprecated, currently in Sept 2023. 
LLM responses can often vary, but the responses may be significantly different when using a different model version.

In [3]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


If you wish to experiment on `LangChain plus platform`:

 * Go to [langchain plus platform](https://www.langchain.plus/) and sign up
 * Create an api key from your account's settings
 * Use this api key in the code below 

In [4]:
#import os
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
#os.environ["LANGCHAIN_API_KEY"] = "..."

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [6]:
question = "What are the diseases discusssed in these documents?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [7]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm.predict("Hello world!")

'Hello! How can I assist you today?'

In [8]:
# Build prompt
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain
from langchain.chains import RetrievalQA

question = "Is cancer discussed in these documents?"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

'No, cancer is not discussed in these documents. Thanks for asking!'

In [9]:
question = "What is the age of onset (typical, min, max)?"
result = qa_chain({"query": question})
result["result"]

'The typical age of onset for C9orf72 frontotemporal dementia and/or amyotrophic lateral sclerosis (C9orf72-FTD/ALS) is usually between 50-64 years. The minimum age of onset can be as early as 20 years, while the maximum age of onset can be as late as 91 years. Thanks for asking!'

### Memory

In [10]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

### ConversationalRetrievalChain

In [11]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()
qa_chain_memory = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [12]:
question = "Is cancer discussed in these documents?"
result = qa_chain_memory({"question": question})
result['answer']

'No, cancer is not discussed in these documents.'

In [13]:
question = "What is the age of onset (typical, min, max)?"
result = qa_chain_memory({"question": question})
result['answer']

"The provided context does not mention anything about cancer or its age of onset. Therefore, I don't have the information to answer your question."